In [ ]:
%load_ext autoreload
%autoreload 2
import waffles
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

from waffles.data_classes.Waveform import Waveform
from waffles.data_classes.WaveformSet import WaveformSet
from waffles.data_classes.UniqueChannel import UniqueChannel
from waffles.data_classes.ChannelWsGrid import ChannelWsGrid
from waffles.utils.numerical_utils import average_wf_ch
from waffles.utils.selector_waveforms import WaveformSelector
from waffles.np02_utils.AutoMap import dict_uniqch_to_module, dict_module_to_uniqch, strUch, ordered_channels_membrane, ordered_channels_cathode
from waffles.np02_utils.PlotUtils import plot_detectors, runBasicWfAnaNP02 
from waffles.np02_utils.PlotUtils import matplotlib_plot_WaveformSetGrid
from waffles.np02_utils.load_utils import open_processed

In [ ]:
run = 39510
dettype = "membrane"
# dettype = "cathode"
nwaveforms = 10000

datadir = f"/eos/experiment/neutplatform/protodune/experiments/ProtoDUNE-VD/commissioning/"
det = "VD_Cathode_PDS" if dettype == "cathode" else "VD_Membrane_PDS"
endpoint = 106 if dettype == "cathode" else 107

dletter = dettype.upper()[0] # C or M...
group1 = [ f"{dletter}{detnum}({chnum})" for detnum in range(1, 3) for chnum in range(1,3) ]
group2 = [ f"{dletter}{detnum}({chnum})" for detnum in range(3, 5) for chnum in range(1,3) ]
group3 = [ f"{dletter}{detnum}({chnum})" for detnum in range(5, 7) for chnum in range(1,3) ]
group4 = [ f"{dletter}{detnum}({chnum})" for detnum in range(7, 9) for chnum in range(1,3) ]
groupLow = group1+group2
groupHig = group3+group4
groupall = group1+group2+group3+group4

In [ ]:
wfset_full = open_processed(run, dettype, datadir, nwaveforms=nwaveforms, mergefiles=True, file_slice=slice(None,None))

In [ ]:
runBasicWfAnaNP02(wfset_full, onlyoptimal=False, baselinefinish=60, int_ll=64, int_ul=100, amp_ll=64, amp_ul=150, configyaml="")

In [ ]:
argsheat = dict(
    mode="heatmap",
    analysis_label="std",
    adc_range_above_baseline=8000,
    adc_range_below_baseline=-1250,
    adc_bins=250,
    time_bins=wfset_full.points_per_wf//2,
    filtering=8,
    share_y_scale=False,
    share_x_scale=True,
    wfs_per_axes=5000,
    zlog=True,
    width=1300,
    height=650*4,
    showplots=True,
    cols=2,
)

#detector = groupLow
# detector = groupHig
detector = groupall

#detector=["M7(1)","M7(2)"]
#detector=["C1(1)"]

plot_detectors(wfset_full, detector, **argsheat)

In [ ]:
extractor = WaveformSelector('cuts.yaml')

In [ ]:
extractor.loadcuts()
wfset_clean = WaveformSet.from_filtered_WaveformSet(wfset_full, extractor.applycuts, show_progress=True)
print(f"Original waveforms: {len(wfset_full.waveforms)}, after cut: {len(wfset_clean.waveforms)}")

In [ ]:

argsheat['adc_range_above_baseline']=8000
argsheat['adc_range_below_baseline']=-2250
argsheat['adc_bins']=400
argsheat['share_y_scale']=True
argsheat['filtering'] = 0
plot_detectors(wfset_clean, detector, **argsheat)

In [ ]:
wfsetch = ChannelWsGrid.clusterize_waveform_set(wfset_clean)[endpoint]

In [ ]:
wfsetch = { k: v for k, v in sorted(wfsetch.items(), key=lambda x: dict_uniqch_to_module[strUch(endpoint, x[0])]) }

In [ ]:
averages = {}
for ch, wfs in wfsetch.items():
    averages[ch] = average_wf_ch(wfs,show_progress=True)
        

In [ ]:
def plot_averages(wfset, averages):
    channel = wfset.waveforms[0].channel
    plt.plot(averages[channel], label=dict_uniqch_to_module[strUch(endpoint, channel)])
    plt.legend()



In [ ]:
matplotlib_plot_WaveformSetGrid(wfset_clean, detector=detector, plot_function=plot_averages,func_params={'averages': averages}, cols=2, figsize=(14, 32))